<a href="https://colab.research.google.com/github/kmrtanmay/pytorch-tutorial/blob/master/pytorch_tutorial/intermediate/bidirectional_rnn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms

In [0]:
#device configuration
device = torch.device('cuda'if torch.cuda.is_available() else 'cpu')

In [0]:
#Hyperparameters

sequence_length = 28
input_size = 28
num_classes = 10
num_layers = 2
num_epochs = 2
hidden_size = 128
batch_size = 100
lr = 0.003

In [0]:
# MNIST dataset
train_set = torchvision.datasets.MNIST(root='../../data/',
                                        train = True,
                                       transform = transforms.ToTensor(),
                                       download=True)
test_set = torchvision.datasets.MNIST(root = '../../data/',
                                      train=False,
                                      transform = transforms.ToTensor(),)

In [0]:
# DataLoader
train_loader = torch.utils.data.DataLoader(dataset=train_set,
                                           batch_size=batch_size,
                                           shuffle=True)
test_loader = torch.utils.data.DataLoader(dataset = test_set,
                                          batch_size = batch_size,
                                          shuffle = False)

In [0]:
# Biderectional recurrent neural network(many-to-one)
class BiRNN(nn.Module):
  def __init__(self,input_size,hidden_size,num_layers,num_classes):
    super(BiRNN,self).__init__()
    self.hidden_size = hidden_size
    self.num_layers = num_layers
    self.lstm = nn.LSTM(input_size,hidden_size,num_layers,batch_first=True,bidirectional=True)
    self.fc = nn.Linear(hidden_size*2,num_classes) # 2 for Bidirectional

  def forward(self,x):
    # set initial sets
    h0 = torch.zeros(num_layers*2,x.size(0),hidden_size).to(device)
    c0 = torch.zeros(num_layers*2,x.size(0),hidden_size).to(device)

    # Forward propagate LSTM
    out,_ = self.lstm(x,(h0,c0))

    # Decode the hidden state of the last time step
    out = self.fc(out[:,-1,:])
    return out

In [0]:
model = BiRNN(input_size,hidden_size,num_layers,num_classes).to(device)

In [0]:
# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(),lr=lr)

In [60]:
#Train Model
total_steps = len(train_loader)
for epoch in range(num_epochs):
  for i,(images,labels) in enumerate(train_loader):
    images = images.reshape(-1,sequence_length,input_size).to(device)
    labels = labels.to(device)

    # Forward pass
    output = model(images)
    loss = criterion(output,labels)

    # Backward propagation and optimize 
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if (i+1)%100 == 0 :
      print('Epoch ({}/{}) : steps ({}/{}) : loss {}'.format(epoch+1,num_epochs,i+1,total_steps,loss.item()))

Epoch (1/2) : steps (100/600) : loss 0.7224975824356079
Epoch (1/2) : steps (200/600) : loss 0.1699470579624176
Epoch (1/2) : steps (300/600) : loss 0.21214410662651062
Epoch (1/2) : steps (400/600) : loss 0.16856706142425537
Epoch (1/2) : steps (500/600) : loss 0.13517731428146362
Epoch (1/2) : steps (600/600) : loss 0.23379196226596832
Epoch (2/2) : steps (100/600) : loss 0.10072837769985199
Epoch (2/2) : steps (200/600) : loss 0.12154239416122437
Epoch (2/2) : steps (300/600) : loss 0.08036773651838303
Epoch (2/2) : steps (400/600) : loss 0.13914139568805695
Epoch (2/2) : steps (500/600) : loss 0.18723738193511963
Epoch (2/2) : steps (600/600) : loss 0.09272439032793045


In [61]:
# Test Model
with torch.no_grad():
  total = 0
  correct= 0
  for i,(images,labels) in enumerate(test_loader):
    images = images.reshape(-1,sequence_length,input_size).to(device)
    labels = labels.to(device)
    output = model(images)
    _,predicted = torch.max(output,1)
    total += labels.size(0) 
    correct += (predicted == labels).sum().item()

  print('Test accuracy of the model on 10000 test images : {} %'.format(100*correct/total)) 


Test accuracy of the model on 10000 test images : 98.09 %


In [0]:
# Save the model checkpoint
torch.save(model.state_dict(),'model.ckpt')